In [1]:
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import pandas as pd
from urllib.request import urlopen
import json
import plotly.express as px

In [2]:
#Example stuff
# Load Data
df = px.data.tips()# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )# Run app and display result inline in the notebook
app.run_server(mode='inline')

# From Google colab

In [3]:
url = "https://github.com/Cpetak/DataLab2021/blob/main/TaskForce1/CountyData.tsv" # Make sure the url is the raw version of the file on GitHub
df = pd.read_html(url)
df=df[0]
data = df.replace('No Data', None)
data = data.replace('#N/A', None)
data.head()

,Unnamed: 0,County,id,rank,education,income,unemployment,disability,life,obesity
0,NaN,"Autauga, Alabama",1001,1371,21.7,53773,6.5,1.6,76.1,38.0
1,NaN,"Baldwin, Alabama",1003,657,27.7,50706,6.8,1,77.7,34.0
2,NaN,"Barbour, Alabama",1005,2941,14.5,31889,11.2,2.9,74.7,47.0
3,NaN,"Bibb, Alabama",1007,2803,9,36824,7.6,2.6,74.2,43.0
4,NaN,"Blount, Alabama",1009,2000,12.4,45192,6.2,1.4,75.9,40.0


In [4]:
net=pd.read_csv("county_network.csv")

In [5]:
from collections import defaultdict

net_map=defaultdict(list)

for indx, row in net.iterrows():
    net_map[row.County_A].append(row.County_B)

In [7]:
data['net']= data['County'].map(net_map)

In [8]:

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [9]:
data['id'] = data['id'].map(lambda x: f'{x:0>5}')

In [12]:
data

,Unnamed: 0,County,id,rank,education,income,unemployment,disability,life,obesity,net,col,wrank
0,NaN,"Autauga, Alabama",01001,1371,21.7,53773,6.5,1.6,76.1,38.0,"[Baldwin, Alabama, Barbour, Alabama, Bibb, Ala...",0,695.375000
1,NaN,"Baldwin, Alabama",01003,657,27.7,50706,6.8,1,77.7,34.0,"[Autauga, Alabama, Barbour, Alabama, Bibb, Ala...",0,455.791667
2,NaN,"Barbour, Alabama",01005,2941,14.5,31889,11.2,2.9,74.7,47.0,"[Autauga, Alabama, Baldwin, Alabama, Bibb, Ala...",0,1380.208333
3,NaN,"Bibb, Alabama",01007,2803,9,36824,7.6,2.6,74.2,43.0,"[Autauga, Alabama, Baldwin, Alabama, Barbour, ...",0,1292.791667
4,NaN,"Blount, Alabama",01009,2000,12.4,45192,6.2,1.4,75.9,40.0,"[Autauga, Alabama, Baldwin, Alabama, Bibb, Ala...",0,929.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134,NaN,"Sweetwater, Wyoming",56037,362,17,72139,4.6,0.5,78.4,35.0,"[Jefferson, Alabama, Maricopa, Arizona, Mohave...",0,339.750000
3135,NaN,"Teton, Wyoming",56039,25,49,69020,6.6,0.2,82.1,21.0,"[Madison, Alabama, Montgomery, Alabama, Apache...",0,116.875000
3136,NaN,"Uinta, Wyoming",56041,674,18.3,59714,5.1,0.8,77.4,36.0,"[Jefferson, Alabama, Matanuska-Susitna Borough...",0,459.291667
3137,NaN,"Washakie, Wyoming",56043,756,23.3,44519,5.4,0.9,77.6,34.0,"[Marshall, Alabama, Yuma, Arizona, Craighead, ...",0,477.416667


# Combining the 2

In [10]:
data["col"]=0

In [13]:
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("US map"),
    dcc.Graph(id='graph'),
    html.Label([
        "Look at migration FROM",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='default', options=[
                {'label': c, 'value': c}
                for c in data.County.unique()
            ]),
        "income",
        dcc.Slider(
            id='income-slider',
            min=0,
            max=1,
            value=0.5,
            step=0.1
            ),
        "education",
        dcc.Slider(
            id='education-slider',
            min=0,
            max=1,
            value=0.5,
            step=0.1
            ),
        "unemployment",
        dcc.Slider(
            id='unemployment-slider',
            min=0,
            max=1,
            value=0.5,
            step=0.1
            ),
        "life",
        dcc.Slider(
            id='life-slider',
            min=0,
            max=1,
            value=0.5,
            step=0.1
            ),
        "disability",
        dcc.Slider(
            id='disability-slider',
            min=0,
            max=1,
            value=0.5,
            step=0.1
            ),
        "obesity",
        dcc.Slider(
            id='obesity-slider',
            min=0,
            max=1,
            value=0.5,
            step=0.1
            )
    ])
])# Define callback to update graph

@app.callback(
    Output('graph', 'figure' ),
    [Input("colorscale-dropdown", "value"), Input('income-slider', 'value'),Input('education-slider', 'value'),Input('unemployment-slider', 'value'),Input('life-slider', 'value'),Input('disability-slider', 'value'),Input('obesity-slider', 'value')]
)
def update_figure(colorscale, income_slider, education_slider, unemployment_slider, life_slider, disability_slider, obesity_slider):
    weight_income = income_slider
    weight_education = education_slider
    weight_unemployment = unemployment_slider
    weight_life  = life_slider
    weight_disability = disability_slider
    weight_obesity = obesity_slider
    

    weighted_data = (1/6)*(weight_income*((data['income'].map(float)).rank(ascending=False)) \
                  + weight_education*((data['education'].map(float)).rank(ascending=False)) \
                  + weight_unemployment*((data['unemployment'].map(float)).rank(ascending=True)) \
                  + weight_life*((data['life'].map(float)).rank(ascending=False)) \
                  + weight_disability*((data['disability'].map(float)).rank(ascending=True)) \
                  + weight_obesity*((data['obesity'].map(float)).rank(ascending=True)))


    data["wrank"]=weighted_data
    data["col"]=0
    
    
    if colorscale != "default":
        
        
        conn=data[data["County"]==colorscale].net.to_numpy()

        for c in conn[0]:
            data.loc[(data.County == c),'col']=1
        
        return px.choropleth(data,geojson=counties,locations='id',color='col', 
        scope='usa',
        color_continuous_scale="Viridis",
        hover_data=["wrank","County"],
        labels={'wrank':'weighted rank'})
    
    return px.choropleth(data,geojson=counties,locations='id',color='wrank', 
    scope='usa',
    color_continuous_scale="Viridis",
    hover_data=["wrank","County"],
    labels={'wrank':'weighted rank'})

app.run_server(mode='inline')